In [3]:
using LinearAlgebra
using GenericLinearAlgebra


##  Objetivo

Dado uma **matriz unitária ou ortogonal** $A \in \mathbb{R}^{(m+n) \times (p+q)}$, queremos fazer a **CS decomposition**:

$$
A = 
\begin{bmatrix}
U_1 & 0 \\
0 & U_2
\end{bmatrix}
\begin{bmatrix}
C & -S & 0 \\
S & C & 0 \\
0 & 0 & I
\end{bmatrix}
\begin{bmatrix}
V_1^\top & 0 \\
0 & V_2^\top
\end{bmatrix}
$$

Esse é o caso mais geral (com bloco identidade no final se sobrarem dimensões), mas vamos focar no **caso simples** para clareza:

### Suponha:

$$
A \in \mathbb{R}^{(m + n) \times r},\quad \text{com } m = n = r
\Rightarrow A \in \mathbb{R}^{2r \times r}
$$

E seja:

$$
A = \begin{bmatrix}
A_1 \\
A_2
\end{bmatrix},\quad A_1, A_2 \in \mathbb{R}^{r \times r}
$$




## Passo a passo da CSD via SVD


### - **Passo 1: Obter os subespaços**

Seja:

* $A_1 \in \mathbb{R}^{r \times r}$
* $A_2 \in \mathbb{R}^{r \times r}$

Você quer decompor $A$ entendendo a **relação geométrica entre os subespaços linha** de $A_1$ e $A_2$.



### - **Passo 2: Aplicar SVD em $A_1$**

Faça:

$$
A_1 = U_1 \Sigma_1 V^\top
$$

* $U_1 \in \mathbb{R}^{r \times r}$ (ortogonal)
* $\Sigma_1 = \text{diag}(\sigma_1^{(1)}, \dots, \sigma_r^{(1)})$
* $V \in \mathbb{R}^{r \times r}$ (ortogonal)


### - **Passo 3: Aplicar SVD em $A_2$** usando **mesmo $V$**

Agora, defina:

$$
\tilde{A}_2 = A_2 V
$$

E aplique SVD em $\tilde{A}_2$:

$$
\tilde{A}_2 = U_2 \Sigma_2
$$

Você agora tem:

* $A_1 = U_1 \Sigma_1 V^\top$
* $A_2 = U_2 \Sigma_2 V^\top$

Essas decomposições **compartilham a mesma base $V$** (isso é importante e possível porque $A$ é unitária).



### - **Passo 4: Obter os ângulos principais**

Agora, queremos achar os ângulos $\theta_i$ tais que:

$$
\cos(\theta_i) = \sigma_i^{(1)}, \quad \sin(\theta_i) = \sigma_i^{(2)}
$$

Como $A$ é unitária:

$$
A_1^\top A_1 + A_2^\top A_2 = I
\Rightarrow \Sigma_1^2 + \Sigma_2^2 = I
$$

Portanto, podemos definir:

$$
\theta_i = \arccos(\sigma_i^{(1)}) = \arcsin(\sigma_i^{(2)})
$$





### - **Passo 5: Construir os blocos da CSD**

* $C = \mathrm{diag}(\cos(\theta_1), \dots, \cos(\theta_r))$
* $S = \mathrm{diag}(\sin(\theta_1), \dots, \sin(\theta_r))$

A matriz do meio da CSD será:

$$
\begin{bmatrix}
C & -S \\
S & C
\end{bmatrix}
$$

E a CSD completa é:

$$
A = 
\begin{bmatrix}
U_1 & 0 \\
0 & U_2
\end{bmatrix}
\begin{bmatrix}
C & -S \\
S & C
\end{bmatrix}
V^\top
$$



##  Recapitulando (Fluxo)

1. **Particione** $A$ em $A_1$ e $A_2$
2. **SVD** de $A_1 = U_1 \Sigma_1 V^\top$
3. Defina $\tilde{A}_2 = A_2 V$, depois faça $\tilde{A}_2 = U_2 \Sigma_2$
4. Calcule $\theta_i = \arccos(\sigma_i^{(1)})$
5. Monte $C, S$, e a decomposição completa



###  Observação importante

Esse processo depende de $A$ ser **unitária/ortogonal** — ou seja, $A^\top A = I$. Caso contrário, essa relação $\Sigma_1^2 + \Sigma_2^2 = I$ **não se sustentaria**, e os cossenos e senos não formariam uma matriz de rotação.



In [192]:
function build_csd_input_matrix(D1::Matrix{Float64}, D2::Matrix{Float64})
    # D1: m × d (m amostras, d features) - dataset 1
    # D2: n × d (n amostras, d features) - dataset 2

    # Passo 1: QR dos transpostos (colunas de D1 e D2 como vetores base)
    Q1, R1 = qr(D1')  
    Q2, R2 = qr(D2') 

    m= size(Q1, 1)
    n = size(R1, 1)
    R1_full = zeros(m, n)
    R1_full[1:n, :] = R1

    m= size(Q2, 1)
    n = size(R2, 1)
    R2_full = zeros(m, n)
    R2_full[1:n, :] = R2

    # Passo 2: Ortonormaliza a união dos subespaços de Q1 e Q2
    Q_all, _ = qr([Q1 Q2])  # Q_all ∈ ℝ^{d × r}, com r ≤ m + n

    # Passo 3: Projeta Q1 e Q2 no subespaço comum Q_all
    A1 = Q_all' * Q1  # ∈ ℝ^{r × m}
    A2 = Q_all' * Q2  # ∈ ℝ^{r × n}
    
    # Passo 4: Constrói a matriz A com blocos
    top = hcat(A1, zeros(size(A1, 1), size(A2, 2)))    # [A1  0]
    bottom = hcat(zeros(size(A2, 1), size(A1, 2)), A2)  # [0   A2]
    A = vcat(top, bottom)  # ∈ ℝ^{2r × (m + n)}

    return A, Q_all, R1_full, R2_full
end

build_csd_input_matrix (generic function with 1 method)

In [213]:
# Exemplo com dados aleatórios
m = 5  # amostras de D1
n = 4  # amostras de D2
d = 6  # número de features

D1 = randn(m, d)  # dataset 1
D2 = randn(n, d)  # dataset 2

A, Q_all, R1, R2 = build_csd_input_matrix(D1, D2)

([1.0 0.0 … 0.0 0.0; 1.3877787807814457e-16 1.0000000000000004 … 0.0 0.0; … ; 0.0 0.0 … -0.3247476540325611 0.5010484629416998; 0.0 0.0 … -0.2840659828266191 -0.1990471101563696], LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}([0.9999999999999998 0.0 … -0.35114782003217804 -0.23836554458478731; -0.021460360930437978 1.0000000000000002 … -0.5784011605749361 0.2511132765007529; … ; -0.08178253529012289 0.20271705798151218 … -0.3247476540325611 0.5010484629416998; -0.338180839114502 -0.1525196757566285 … -0.2840659828266191 -0.1990471101563696], [1.5112106498199231 -0.5725600652508731 … 0.3156226098607736 0.0; 0.0 1.4510135652547944 … -0.29851632112516374 0.0; … ; 0.0 0.0 … 1.4424387311495952 0.0; 0.0 0.0 … 0.0 0.0]), [-2.3226321744898404 -0.09462615009086806 … 0.6688532523827969 -0.1916376576048246; 0.0 2.049094425183275 … 0.09975038600374153 0.18625272833176093; … ; 0.0 0.0 … 0.0 0.6804424022938356; 0.0 0.0 … 0.0 0.0], [-2.0351156141143303 0.5243766085794078 0.657

In [214]:
display(R1)
display(R2)
display(Q_all)
display(A)

6×5 Matrix{Float64}:
 -2.32263  -0.0946262   0.284915   0.668853   -0.191638
  0.0       2.04909    -0.861688   0.0997504   0.186253
  0.0       0.0        -0.600405  -1.23307    -0.936787
  0.0       0.0         0.0       -1.38027    -0.425887
  0.0       0.0         0.0        0.0         0.680442
  0.0       0.0         0.0        0.0         0.0

6×4 Matrix{Float64}:
 -2.03512  0.524377   0.657513  -0.610833
  0.0      1.02771   -1.33996   -0.593236
  0.0      0.0        2.39095    0.157337
  0.0      0.0        0.0       -1.81606
  0.0      0.0        0.0        0.0
  0.0      0.0        0.0        0.0

6×6 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}

12×12 Matrix{Float64}:
 1.0           0.0          -1.80411e-16  …   0.0         0.0        0.0
 1.38778e-16   1.0           6.66134e-16      0.0         0.0        0.0
 1.11022e-16  -1.11022e-16   1.0              0.0         0.0        0.0
 5.55112e-17  -5.55112e-17   0.0              0.0         0.0        0.0
 2.77556e-17  -1.11022e-16   1.66533e-16      0.0         0.0        0.0
 0.0          -8.32667e-17   1.11022e-16  …   0.0         0.0        0.0
 0.0           0.0           0.0             -0.307233   -0.351148  -0.238366
 0.0           0.0           0.0             -0.0756162  -0.578401   0.251113
 0.0           0.0           0.0              0.505592    0.488782  -0.179462
 0.0           0.0           0.0              0.371695   -0.342177  -0.74649
 0.0           0.0           0.0          …   0.692064   -0.324748   0.501048
 0.0           0.0           0.0              0.164796   -0.284066  -0.199047

In [215]:
function rebuild_datasets_matrixes(A, Q_all, R1, R2)
    # A: matriz de entrada
    # Q_all: base ortonormal comum
    # R1: matriz triangular superior de D1
    # R2: matriz triangular superior de D2

    r = size(Q_all, 2)  # dimensão do subespaço comum

    A1 = A[1:r, 1:size(R1, 1)] 
    D1_reconstructed = (Q_all * (A1' \ R1))'
   
    A2 = A[r+1:end, end-size(R2,1)+1:end] 
    D2_reconstructed = (Q_all * (A2' \ R2))'
    
    # D1_reconstructed: matriz reconstruída de D1
    # D2_reconstructed: matriz reconstruída de D2

    # Retorna as matrizes reconstruídas
    return D1_reconstructed, D2_reconstructed
    
end

rebuild_datasets_matrixes (generic function with 1 method)

In [220]:
D1_reconstructed, D2_reconstructed = rebuild_datasets_matrixes(A, Q_all, R1, R2)
display(D1)
display(D2)
display(D1_reconstructed)
display(D2_reconstructed)
if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
    
end

5×6 Matrix{Float64}:
  1.18735   -0.0753256  -1.20869    -1.01331    -0.287056  -1.18701
  1.28806   -0.953842    0.583808    0.886238   -0.715811  -0.014116
 -0.593211   0.734436   -0.0282487   0.0450226   0.495124  -0.211899
  0.540176   0.839112    0.133372    1.1492      1.18056    0.393497
  0.319066   0.341549   -0.520077    0.924657    0.289179  -0.407201

4×6 Matrix{Float64}:
  0.0394613  -1.87577     0.737148   0.0808812    0.191689   0.186914
 -0.750263    0.449168   -0.295731   0.227416     0.468467  -0.455922
  1.89319     0.394466   -0.17961   -0.854753    -0.852075  -1.64797
  1.08804     0.0380192   1.45625   -0.00223514   0.837456   0.201559

5×6 adjoint(::Matrix{Float64}) with eltype Float64:
  1.18735   -0.0753256  -1.20869    -1.01331    -0.287056  -1.18701
  1.28806   -0.953842    0.583808    0.886238   -0.715811  -0.014116
 -0.593211   0.734436   -0.0282487   0.0450226   0.495124  -0.211899
  0.540176   0.839112    0.133372    1.1492      1.18056    0.393497
  0.319066   0.341549   -0.520077    0.924657    0.289179  -0.407201

4×6 adjoint(::Matrix{Float64}) with eltype Float64:
  0.0394613  -1.87577     0.737148   0.0808812    0.191689   0.186914
 -0.750263    0.449168   -0.295731   0.227416     0.468467  -0.455922
  1.89319     0.394466   -0.17961   -0.854753    -0.852075  -1.64797
  1.08804     0.0380192   1.45625   -0.00223514   0.837456   0.201559

As matrizes reconstruídas são equivalentes às originais!
